In [1]:
# %% Import Libraries and Data

# Third-party imports
import numpy as np
import pyansys

# Local imports
from util_loads import Propeller, Airfoil, Loadcase
from util_mapdl import Material
from femodel import Threepartmodel

# %% Instantiate Airfoils and assign radial sections

airfoil = Airfoil('mf3218.xfo', 500000, iter_limit=600)
airfoildick = Airfoil('mf3218-dick.xfo', 500000, iter_limit=600)
rectangle = Airfoil('rectangle2.txt',500000)

# %% Instantiate Propeller and assign geometry and airfoils

propeller = Propeller(number_of_blades=2,
                      tip_radius=0.412,
                      hub_radius=0.04,
                      )

propeller.geometry = np.array([[0.10,0.078,0],
                               [0.121, 0.078, 0.],
                               [0.155, 0.100, 5.99],
                               [0.223, 0.160, 17.97],
                               [0.345, 0.149, 14.44],
                               [0.417, 0.142, 12.68],
                               [0.490, 0.135, 11.18],
                               [0.563, 0.128, 9.94],
                               [0.636, 0.121, 8.97],
                               [0.709, 0.114, 8.26],
                               [0.782, 0.107, 7.81],
                               [0.854, 0.100, 7.63],
                               [0.947, 0.091, 7.5],
                               [1., 0.066, 7.5],                               
                               ])

propeller.sections = [[0.121, airfoil],
                      [0.223, airfoil],
                      [1., airfoil]]

propeller.geometric_sections = [[0.121, rectangle],
                                [0.223, airfoildick],
                                [1., airfoildick]]

for x in propeller.sections:
    x[1].set_polar(alpha_start=-7, alpha_stop=20, alpha_inc=0.25) 
    
airfoil.xrotor_characteristics['Cm'] = -0.14
airfoil.xrotor_characteristics['d(Cl)/d(alpha)'] = 6.28
airfoil.xrotor_characteristics['Minimum Cl'] = 0.

# %% Instantiate Loadcases

propeller.add_loadcase(loadcase=Loadcase(name='Max RPM', flight_speed=0.01))

propeller.loadcases[0][0].set_data('rpm',4000)

# %% Calculate loads

propeller.calc_loads()
propeller.set_load_envelope()

In [3]:
propeller.loadcases[0][1]

{'single_values': {'no. blades': 2.0,
  'thrust(N)': 175.0,
  'Efficiency': 0.0007,
  'Eff induced': 0.0008,
  'Tnacel(N)': -0.0,
  'Tvisc(N)': -0.2482,
  'rho(kg/m3)': 1.226,
  'radius(m)': 0.412,
  'power(W)': 2530.0,
  'speed(m/s)': 0.01,
  'Eff ideal': 0.0009,
  'hub rad.(m)': 0.04,
  'Pvisc(W)': 266.0,
  'Vsound(m/s)': 340.0,
  'adv. ratio:': '0.00006',
  'torque(N-m)': '6.05',
  'rpm': '4000.000',
  'Tcoef': '********',
  'disp. rad.': '0.0400',
  'mu(kg/m-s)': '1780E-04'},
 'oper':      i    r/R     c/R  beta(deg)     CL      Cd  REx10^3   Mach   effi   effp  \
 0    1  0.104  0.0778      -0.06  0.872  0.0206    42.10  0.056 -0.007  1.412   
 1    2  0.126  0.0793       0.40  0.895  0.0179    51.13  0.067 -0.007  1.440   
 2    3  0.163  0.1067       7.53  1.029  0.0181    83.92  0.082  0.011  0.647   
 3    4  0.206  0.1483      15.87  1.083  0.0170   143.01  0.100  0.002  0.912   
 4    5  0.252  0.1684      19.25  1.077  0.0147   199.55  0.123  0.001  0.941   
 5    6  0.299 